In [1]:
import tensorflow as tf
import pandas as pd
import keras
import os

Using TensorFlow backend.


# Data Processing, Loading

In [2]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [3]:
import pandas as pd


dirName = 'data';
    
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
    

df=pd.DataFrame(columns=["Title","Description","Category"])
# Print the files    

for elem in listOfFiles:
    file1 = open(elem,"r") 
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    """
    print("Category: "+elem.split("/")[1])
    print("Title:"+sampleNews[0])
    print("Description:"+newsDesc[0:50])
    """
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[sampleNews[0],newsDesc,elem.split("/")[1]]])
    df=df.append(dfsample)

In [4]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL\'s existing customers for high-spee

In [5]:
df.head()

,Title,Description,Category
0,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarner...,business
1,Dollar gains on Greenspan speech,The dollar has hit its highest level against t...,business
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuko...,business
3,High fuel prices hit BA's profits,British Airways has blamed high fuel prices fo...,business
4,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domec...,business


In [6]:
categories=df["Category"].unique().tolist()
cat_dict=dict(zip(categories, range(0,len((categories)))))
print(cat_dict)
for x in categories:
    print(x+": "+str(len(df[df["Category"]==x])))
df["Category"]=df['Category'].map(cat_dict, na_action='ignore')

{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
business: 510
entertainment: 386
politics: 417
sport: 511
tech: 401


In [7]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [23]:
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]", " ")
df=df.dropna()
df["Description"].head()[0]

'Quarterly profits at US media giant TimeWarner jumped     to      bn       m  for the three months to December  from     m year earlier The firm  which is now one of the biggest investors in Google  benefited from sales of high speed internet connections and higher advert sales  TimeWarner said fourth quarter sales rose    to      bn from      bn  Its profits were buoyed by one off gains which offset a profit dip at Warner Bros  and less users for AOL Time Warner said on Friday that it now owns    of search engine Google  But its own internet business  AOL  had has mixed fortunes  It lost         subscribers in the fourth quarter profits were lower than in the preceding three quarters  However  the company said AOL s underlying profit before exceptional items rose    on the back of stronger internet advertising revenues  It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL s existing customers for high speed 

# Data Cleaning

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df["Description"], df["Category"], test_size=0.20, random_state=42)

train_sum=0
for x in categories:
    print(x+": "+str(len(y_train[y_train==cat_dict[x]])))
    train_sum+=len(y_train[y_train==cat_dict[x]])
print("Total Training Data: "+str(train_sum)+"\n")

test_sum=0
for x in categories:
    print(x+": "+str(len(y_test[y_test==cat_dict[x]])))
    test_sum+=len(y_test[y_test==cat_dict[x]])
print("Total Test Data: "+str(test_sum))

business: 395
entertainment: 314
politics: 341
sport: 409
tech: 321
Total Training Data: 1780

business: 115
entertainment: 72
politics: 76
sport: 102
tech: 80
Total Test Data: 445


### Tokenization

In [10]:
import nltk
from nltk.tokenize import word_tokenize

X_train_notcleaned=[word_tokenize(i) for i in X_train]
y_train=y_train.values
X_test_notcleaned=[word_tokenize(i) for i in X_test]
y_test=y_test.values

print(X_train_notcleaned[0])

['Ashley', 'Cole', 'has', 'refused', 'to', 'blame', 'Robin', 'van', 'Persie', 'for', 'leaving', 'Arsenal', 'with', 'no', 'fully', 'fit', 'strikers', 'for', 'the', 'FA', 'Cup', 'fifth', 'round', 'replay', 'at', 'Sheffield', 'United', 'Van', 'Persie', 'is', 'suspended', 'alongside', 'Dennis', 'Bergkamp', 'and', 'Jose', 'Antonio', 'Reyes', 'after', 'being', 'sent', 'off', 'at', 'Southampton', 'when', 'Arsenal', 'had', 'a', 'numerical', 'advantage', 'Thierry', 'Henry', 'is', 'ruled', 'out', 'with', 'an', 'Achilles', 'tendon', 'injury', 'but', 'Cole', 'said', 'No', 'one', 'is', 'putting', 'the', 'blame', 'on', 'Robin', 'It', 's', 'just', 'something', 'that', 'happens', 'on', 'the', 'spur', 'of', 'the', 'moment', 'Cole', 'added', 'I', 've', 'done', 'it', 'before', 'and', 'I', 'hope', 'they', 'didn', 't', 'blame', 'me', 'for', 'anything', 'Of', 'course', 'he', 'll', 'learn', 'I', 've', 'been', 'sent', 'off', 'a', 'couple', 'of', 'times', 'now', 'and', 'it', 's', 'just', 'one', 'of', 'those', 

In [12]:
from nltk.corpus import stopwords

X_train,X_test=[],[]
for s in X_train_notcleaned:
    s_cleaned = []
    for w in s:
        if w not in stopwords.words("english") and len(w)>=3:
            s_cleaned.append(w)
 
    X_train.append(s_cleaned)
for s in X_test_notcleaned:
    s_cleaned = []
    for w in s:
        if w not in stopwords.words("english") and len(w)>=3:
            s_cleaned.append(w)
 
    X_test.append(s_cleaned)
print(X_train[0])

['Ashley', 'Cole', 'refused', 'blame', 'Robin', 'van', 'Persie', 'leaving', 'Arsenal', 'fully', 'fit', 'strikers', 'Cup', 'fifth', 'round', 'replay', 'Sheffield', 'United', 'Van', 'Persie', 'suspended', 'alongside', 'Dennis', 'Bergkamp', 'Jose', 'Antonio', 'Reyes', 'sent', 'Southampton', 'Arsenal', 'numerical', 'advantage', 'Thierry', 'Henry', 'ruled', 'Achilles', 'tendon', 'injury', 'Cole', 'said', 'one', 'putting', 'blame', 'Robin', 'something', 'happens', 'spur', 'moment', 'Cole', 'added', 'done', 'hope', 'blame', 'anything', 'course', 'learn', 'sent', 'couple', 'times', 'one', 'things', 'bit', 'crazy', 'one', 'two', 'seconds', 'Freddie', 'Ljungberg', 'likely', 'used', 'emergency', 'striking', 'role', 'partnered', 'either', 'Arturo', 'Lupoli', 'Quincy', 'Owusu', 'Abeyie', 'Jeremie', 'Aliadiere', 'Gunners', 'boss', 'Arsene', 'Wenger', 'said', 'Freddie', 'option', 'need', 'second', 'striker', 'decide', 'whether', 'Aliadiere', 'Quincy', 'Lupoli', 'start', 'front', 'Those', 'three', 'in

In [13]:
words = set([])
for s in X_train:
    for w in s:
        words.add(w.lower())
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

In [14]:
train_sentences_X, test_sentences_X = [], []

EMB_DIM=300
num_words=len(word2index)+1
print("Number of Words:"+str(num_words))

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
    
for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)


MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
print(train_sentences_X[0])
print(y_train[0])

Number of Words:25132
1847
[11704  3187 23251 ...     0     0     0]
3


In [18]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout
from keras.optimizers import Adam
from keras.models import load_model

model=Sequential() 
#embedding_layer=Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_LENGTH,trainable=True)
embedding_layer=Embedding(num_words, 300,mask_zero=True)
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(embedding_layer)
model.add(Bidirectional(LSTM(128,activation="tanh")))
model.add(Dropout(0.3))
model.add(Dense(1,activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1847, 300)         7539600   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 7,979,153
Trainable params: 7,979,153
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
es = EarlyStopping(monitor='val_acc', mode='max',verbose=1,patience=3,min_delta=0.003)
history=model.fit(train_sentences_X, y_train, validation_data=(test_sentences_X,y_test), batch_size=64,epochs=15,callbacks=[es,tbCallBack])

Train on 1780 samples, validate on 445 samples
Epoch 1/15
 100/1780 [>.............................] - ETA: 7:27 - loss: nan - acc: 0.1500

KeyboardInterrupt: 